<h2><center><strong>Uniquant Market Prediction -  Team B</strong></center></h2>
<h3><center><strong>Introduction</strong></center></h3>

In financial market, fluctuation are expected irrespective of the ivestment strategy used. However, investors still find ways to estimate their overall returns. Factors including the ivestment types have different risks and give different returns, which significantly impact the stability and volatility. There are many computer-based algorithms and models in financial market trading used to predict returns.

Ubiquant Investment (Beijing) Co., Ltd, one of the leading domestic quantitative hedge fund based in China was established in 2012. To drive quantitative financial market investment, Ubiquant rely on international talents in math and computer science together with cutting-edge technology. In general, Ubiquant is committed to creating long-term stable returns for investors.

<h3><center><strong>Problem Statement</strong></center></h3>
The goal of Ubiquant Market Competition is to predict the future return of an asset given features over a specific period. All the elements in the dataset are anonymized by applying a function which simulate the actual dataset. The period over which the observations were recorded is concealed. 
The dataset consists of 1211 different time steps and 3500 assets. The time steps in which investment features were recorded differed, and some got less than two records. Setting up a prediction model out of this is challenging as 27 percent of information is missing. Filling the gaps with an interpolation algorithm is difficult as well. The targets or forward returns show a normal distribution, and do not appear to correlate with features. It is noteworthy that there are so many correlated features.
There are three ways to address this problem, and first is handling it as a regression task by predicting the targets using existing features. Second is factoring time and features to forecast targets or a combination of the first and second technique. 

<h3><center><strong>Goal</strong></center></h3>

The main goal of the competition is to build a model that forecasts an investment's return rate. In both the training and testing, the algorithm utilizes historical prices data. A well developed model will solve the real-world data science problem on return forecasting with a higher accuracy possible. 

This will improve the ability of quantitative researchers to forecast returns. In addition, it will enable investors at any scale to make better decisions.


<h3><center><strong>Metrics</strong></center></h3>
Evaluation of the model is based on the mean of the Pearson correlation coefficient for each time ID. The submistion included the use of a time-series API, to ensures the model does not peek forward in time. An error could occur if the submission includes nulls or infinities and the submissions that only include one prediction value would receive a score of -1.

Since this is a forecasting competition with an active training phase,their will be a second period where models will be run 

This will improve the ability of quantitative researchers to forecast returns. In addition, it will enable investors at any scale to make better decisions.
<h3><center><strong>Methodology</strong></center></h3>

The first task of this challenge was modelling a minimum viable product that would predict the targets based on existing features. Team members agreed to work on this task individually, and make a submission. The final model for submission was based on the performances of the models, it's complexity, computational demands and difficulty.

<h4><strong>Random Forest Regressor - Rhodasi</strong></h4>

A Random Forest Regressor was implemented using Keras module in Tensorflow. The model is best suited for classification tasks with multiple features. More so Random Forest Regressor is robust to outliers, noise and variance. The model was trained on 30 percent of the dataset and achieved and  accuracy of 0.021. Due to the size of the dataset, the tree depth is limited by compute resources which reduces model performance.

<h4><strong>Vector Autoregression - Lali</strong></h4>

<h4><strong>A DNN with Leaky ReLu - Mercelina</strong></h4>
Deep learning models are able to automatically learn arbitrary complex mappings from inputs to outputs.

A deep learning model with LeakyReLU activation was used as an MVP. During training, loss optimization was Adam with a learning rate of 0.003. The training was conducted for 5 epochs and gave a score of 0.142.  Maintaining 5 epochs with a learning rate of 0.001 improved the score to 0.143. Adjusting the learning rate to 0.001, and training epochs to 20, a score of 0.146 was attained. Further increase in epochs to 30, lead to increase in validation loss due to overfiting, and the score reduced to 0.144. Using early stopping and saving the best model did not improve the model score from 0.146.

<h4><strong>A Dense Convolution Neural Network -  Dennies</strong></h4>

A convolution layer as a feature engineering technique was used to reduce dimensionality. A single channel image of 1 by 300 dimensions is the input to the network. The convolution layer has a 3 by 3 filter, and max pooling layers with a stride of 2. The output of the layer is then fed into a dense layer. The convolution layer reduces the input dimensions while retaining key features.
The data set was split into five folds, and which were then further subdivided into the training and validation set. During training, early stopping was used to decrease over-fitting when validation loss rose. The root mean square of the training set was 0.91 and the Pearson score was 0.139 on the leaderboard.
Considering the computational demands and low performance despite various optimization techniques, the model was discarded for the other fronted models.
<h3><center><strong>Selected Model - </strong></center></h3>
Team members evaluated their mvps and agreed on working with Arnold's model based on dense neural network and ensemble learning. The leaderboard score was 0.149, outperforming the other fronted models. The model employs ensemble techniques which allows for improvement. The team worked on the model to improve its performance and achieved a maximum score of 0.153.
The next section breaks down into the EDA, feature engineering, model training and submission



## **Exploratory Data Analysis**
We explore the structure of the data, its characteristics, and relationships

In [2]:
import os

# exploratory data analysis
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import matplotlib as mpl
from itertools import cycle
from scipy import stats
import seaborn as sns

# model training
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops

# feature engineering


plt.style.use("ggplot")
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

### **Load the Dataset**.
<p> The pickle file is adapted from <a href="https://www.kaggle.com/lonnieqin/ubiquant-market-prediction-half-precision-pickle"><strong>Lonnie</strong></a> which is a reduced version of the original dataset. This speeds up loading and minimizes memory exhaustion</p>

In [3]:
# train df
train = pd.read_pickle('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl')

# convert the train as float16

train = train.astype('float16')

In [ ]:
train.head(5)

The train dataframe has 300 anonymized features labelled f_0 to f_299, investments and targets at different times.

In [ ]:
#  extracting the features of the df
n_features = 300
features = [f'f_{i}' for i in range(n_features)]

#### **Are there any nulls? What are the total number of observations?**
<a id="1"></a>

In [ ]:
#  check the shape and NaNS

obs = train.shape[0]
print(f"number of observations: {obs}")
print(f"NaNs in the dataframe? {str(train.isnull().values.any())}")

### **Investments & Observation size (Time)**
<a id='time_investments'></a>

All the investments have a total of 3141410 observations from the [cell](#1) above. We investigate if these observations are distributed equally among the investments. 

In [ ]:
# the number of investment ids, and max available time ids
# the investment_id with highest number of timesteps

obs_by_asset = train.groupby(['investment_id'])['target'].count()
# number of unique investments
assets = train.investment_id.nunique()
time_steps =train.time_id.nunique()
print(f"number of investments: {assets}\nmax time steps: {time_steps}")
print(f"highest observation window: investement_id {np.argmax(obs_by_asset)} with {np.max(obs_by_asset)}")
print(f"lowest observation window: investement_id {np.argmin(obs_by_asset)} with {np.min(obs_by_asset)}")


There  are 3579 investments. Investment 1334 has the lowest observation time of 2 whereas investment 2027 has highest observation time of 1211.
Investment have different observation times and therefore the dataset is not balanced.
Some of the investment_id's are underepresented in the datset. We may attempt to remove them to see if our model's accuracy and/or predictive power will improve

#### **Quartile Distribution**

In [ ]:
# finding the quartiles of the time steps 

obs_by_asset.quantile([0.25,0.5,0.75])



50% of the investments have time steps between 0 and 683 for lower quartile and 1131 and 1211 for Q3.
The rest are between median value of 1009 and Q3

In [ ]:
# visual rep of the sizes of observations

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
obs_by_asset.plot.hist(bins=60)
plt.title("timesteps by asset distribution")
plt.show()

The sample size of each investment is equal to the time steps used in recording the data. As computed in the previous cell, the majority of the investments have observations between 800 and 1200
<a id="2"></a>

In [ ]:
# we investigate the distribution of each investment observation across the min and the max time steps

train[['investment_id', 'time_id']].plot.scatter('time_id', 'investment_id', figsize=(20, 30), s=0.5)
plt.show()

The white patches shows missing data at the specific time id. The are some vertical white lines which are common across the various observations. These are probably holidays, missing or removed data.

### **Targets && Investment Ids**
<a id='targets_investments'></a>

We average the targets of each investment across the observation window. For each investment also, the standard deviation of targets is calculated.

In [ ]:
# The mean of the targets of each asset
mean_target = train.groupby(['investment_id'])['target'].mean()

# mean of the mean of each investment
mean_mean_target =mean_target.mean()
kde = stats.gaussian_kde(mean_target)

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
mean_target.hist(bins=60)
plt.title("mean target distribution")
plt.show()

print(f"Mean of mean target: {mean_mean_target: 0.5f}")

The distribution of targets is normal with a mean of almost zero. There are no outliers causing long tails, and can be used for modelling without further preprocessing

In [ ]:
# the standard deviation of the tragets of each investment id

sts_target = train.groupby(['investment_id'])['target'].std()

# the mean of the std of the targets
mean_std_target = np.mean(sts_target)

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
sts_target.plot.hist(bins=60)
plt.title("standard deviation of target distribution")
plt.show()

print(f"Mean of std target: {mean_std_target: 0.5f}")

The targets have a mean standard deviation of 0.9. Approximately 95% of the investments have a mean std between 0.5 and 1.5. There are a few data points with high STD

In [ ]:
#  A plot of the std and the mean of each each asset
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
sts_target.plot(style='.-',color=next(color_cycle),legend=True,label='MEAN_STD')
ax.axhline(y=mean_std_target,linestyle='--',color=next(color_cycle))
ax.text(4000,mean_std_target+0.1,'mean std targets')

ax.text(4000,mean_mean_target+0.1,'mean mean targets')
mean_target.plot(style='.-',color=next(color_cycle),legend=True,label='MEAN_MEAN')
ax.axhline(y=mean_mean_target,linestyle='--',color=next(color_cycle))
plt.title("standard deviation & Mean of target distribution")
plt.show()

The plot of mean of asset targets is about zero, and confirm distribution of targets as normal. There a few cases of spiking in STD, and probably the reason for outliers in the distribution plot.

### **Time and Targets**
In this section, we are utilizing the observation size of each investment from [Investments & Time](#time_investments) and [Investment Targets](#targets_investments) to visualize the relationship of the time and the targets. We use Seaborn Joint plot library to compare how the mean and standard deviation of each investment target vary across time

In [ ]:
#  obs by target is a count of the time steps per investment id
#  A plot of mean of targets for each investment id

ax = sns.jointplot(x=obs_by_asset.values, y=mean_target, kind="reg", 
                   height=8, joint_kws={'line_kws':{'color':next(color_cycle)}})
ax.ax_joint.set_xlabel('count timesteps/investment')
ax.ax_joint.set_ylabel('mean target of investment')
plt.show()

The mean is constant for all the time ids. 

In [ ]:
#  the count of investment time steps against mean std of each investment

qx = sns.jointplot(x=obs_by_asset.values, y=sts_target, kind="reg", 
                   height=8, joint_kws={'line_kws':{'color':next(color_cycle)}})
ax.ax_joint.set_xlabel('observations')
ax.ax_joint.set_ylabel('std target')
plt.show()

Time affects the mean standard deviation of the targets. It's obvious that the standard deviation reduces with increasing time steps
This is in contrast to the mean which is constant through the observation space.

#### **Is there any correlation between time and targets?**

In [ ]:
#  we find the correlation of the time id and the targets

r = np.corrcoef(train.groupby('time_id')['investment_id'].nunique(), train.groupby('time_id')['target'].mean())[0][1]

print(f"Correlation of number of assets by target: {r:0.3f}")

The coefficient is negative, and almost zero.Therefore, it can be concluded that there is no correlation.
In the next section, we will plot the mean and std of all the investments at different time windows.

In [ ]:
time2target_mean = train.groupby(['time_id'])['target'].mean() #mean of targets at specific time ids
time2target_std = train.groupby(['time_id'])['target'].std()  #std of targets at each time id

_, axes = plt.subplots(1, 1, figsize=(24, 12))

# fills the region between the minimum mean and maximum mean of the investments
plt.fill_between(
        time2target_mean.index,
        time2target_mean - time2target_std,
        time2target_mean + time2target_std,
        alpha=0.1,
        color="b",
    )

plt.plot(
        time2target_mean.index, time2target_mean, "o-", color=next(color_cycle), label="Training score"
    )
plt.axhline(y=mean_mean_target, color=next(color_cycle), linestyle='--', label="mean")
plt.title("Mean Targets vs TimeIDs")
axes.set_ylabel("target")
axes.set_xlabel("time")
plt.show()

There is spiking in average of the mean between time id 380 to 500 compared to other regions where the target mean appear to be uniform. In this region, probably, there are missing data points for the majority of the investment ids. The white patches in the scatter plot [here](#2) concides with the spikes in this plot. It's the evidence of missing data.

In [ ]:
# a random investment id is 
time2target_mean = train.groupby(['time_id'])['target'].mean()
time2target_std = train.groupby(['time_id'])['target'].std()

_, axes = plt.subplots(1, 1, figsize=(24, 12))
plt.fill_between(
        time2target_mean.index,
        time2target_mean - time2target_std,
        time2target_mean + time2target_std,
        alpha=0.1,
        color="b",
    )
plt.plot(
        time2target_mean.index, time2target_mean, "o-", color=next(color_cycle), label="time-target mean"
    )
plt.axhline(y=mean_mean_target, color=next(color_cycle), linestyle='--', label="mean")

asset = np.random.randint(0,train.investment_id.nunique())
plt.scatter(train[train.investment_id==asset].time_id,
               train[train.investment_id==asset].target, color=next(color_cycle),label=f'investment_id {asset}')
plt.legend()

axes.set_ylabel("target")
axes.set_xlabel("time")
plt.show()

A random investment is scattered along the mean-target-time plot. For the next section, we inspect specific investments, and how time affects it's targets

In [ ]:
# we are plotting the random investment ids against time ids

random_investments = set(list(np.random.randint(0,300) for num in range(7)))

for investment_id in random_investments:
    d = train.query('investment_id == @investment_id')
    d.set_index('time_id')['target'] \
        .plot(figsize=(15, 5),
              title=f'Investment_id {investment_id}',
              color=next(color_cycle),
              style='.-')
    plt.show()

The plots of random assets indicate missing data and gaps in the observation window. Some of them such as investment 262 has completely no data points. The missing points will impede model performance in the training section. Investment 109 is selected to demonstrate these data gaps

In [ ]:
from matplotlib.patches import Ellipse
# a plot of investment_id 59
investment_id=109

fig, ax = plt.subplots(1, 1,figsize=(15, 5))
d = train.query('investment_id == @investment_id')

d.set_index('time_id')['target'] \
    .plot(title=f'Investment_id {investment_id}',
          color=next(color_cycle),
          style='.-')

circle_rad = 30  # This is the radius, in points
circle_2 = 40
color = next(color_cycle)
ax.plot(380, 0.5, 'o',
        ms=circle_rad * 2, mec=color, mfc='none', mew=2)
ax.annotate('Missing Data', xy=(380,1), xytext=(400, 4),
            color='b', size='large',
            arrowprops=dict(
                arrowstyle='simple,tail_width=0.3,head_width=0.8,head_length=0.8',
                facecolor=color, shrinkB=circle_rad * 0.15))
ax.plot(1100, 1.2, 'o',
        ms=circle_2 * 2, mec=color, mfc='none', mew=2)
ax.annotate('', xy=(1100,1.2), xytext=(450, 4),
            color='b', size='large',
            arrowprops=dict(
                arrowstyle='simple,tail_width=0.3,head_width=0.8,head_length=0.8',
                facecolor=color, shrinkB=circle_2 * 0.05))

plt.show()

### **Features**

There are 300 features whose values are anonymized. In the first section, we will plot the distribution of 30 features 

In [ ]:
def plot_hist():
    '''Plot distribution of the features extracted randomly
    between 0 & 299
    
    '''

    columns = [f"f_{_}" for _ in np.random.choice(range(299), 30, replace=False)]
    
    train.hist(column = columns, bins = 100, figsize = (30,30))
    plt.show()

    return

plot_hist()


Its difficult to make sense of the histograms of the features. There are some which exhibit a normal distribution, some which are highly skewed with long tails, and multimodal(non-uniform) distribution.
What is conspicous also is missing data points.

#### **features vs targets**
>First, is trending the random features vs targets to check for trends or correlation
<a id='3'></a>

In [ ]:
#  Using 3M+ observations will cause memory lag. Observations are sampled

target_sample = train.sample(frac = 0.10)

def target_feature():
    
    """
    This function randomly picks 30 features without replacement,
    and plot against the target.
    """
    
    fig, ax = plt.subplots(6, 5, figsize=(30,30), sharey=True)
    
    axes = [axis for axis in ax.flatten()]
    
    for i, _ in enumerate(np.random.choice(range(299), 30, replace=False)):
        ax_ = axes[i]

        df_ = target_sample[['target',f'f_{_}']]
        ax_.scatter(df_[f'f_{_}'], df_['target'], alpha=0.15)
        ax_.get_xaxis().set_visible(False)
        ax_.get_yaxis().set_visible(False)
        ax_.axis('off')
        plt.text(
            0.1,
            0.9,
            f'f_{_}',
            horizontalalignment='left',
            verticalalignment='top',
            transform = ax_.transAxes)
        del df_
    plt.show()

# target_feature()

The scatter plots appear as noise, and there seems to be no visible trends among the features and targets. We are then going to compute the correlation of the features and targets.

In [ ]:
# correlation of features && targets

correlation = target_sample[['target']+features].corr()

In [ ]:
# 
fig, ax = plt.subplots(1,1,figsize=(10, 4))

plt.hist(correlation["target"][1:], edgecolor = "black", bins = 25)
ax.set_ylabel("Frequency of  Corr")
ax.set_xlabel("Target-Feature Correlation")
plt.show()

There seems to be a no strong linear correlation between the features and the target. Max positive corr is 0.04, and the min is -0.06. Feature-target plots [here](#3) had shown no trend  between the targets and features.

In [ ]:
sns.clustermap(abs(correlation), figsize = (15,15), cmap = "mako");

Looking at the linear correlation between the fx features, we can see that there are several highly correlated features. Highly correlated features often contain the same information and we could drop some features without information loss.

##### **Feature importance using correlations**

In [ ]:
#Check if any of the features are at least moderately correlated with the target using pearson's r <-0.5 or >0.5
correlations=[]
feature_columns=[]

for col in train.columns[2:-1]:
    coef = np.corrcoef(train['target'], train[col])[0][1]
    correlations.append(coef)
    feature_columns.append(col)
    
target_feature_coeff=pd.concat([pd.Series(feature_columns), pd.Series(correlations)],axis=1)

#rename the columns in our correlations_target df
target_feature_coeff['features']=target_feature_coeff[0]
target_feature_coeff['coeff']=target_feature_coeff[1]
target_feature_coeff.drop([0,1], axis=1)

#visualize the importance of features based on pearson's r
i = target_feature_coeff['features']
feat_importances = pd.Series(correlations, index=i)
feat_importances.nlargest(100).plot(kind='barh', figsize=(20, 15)).invert_yaxis()
plt.show()

### **Checking whether the data is static or not**
We are testing whether the data is stationary or not using the Augmented Dickey-Fuller test (ADF Test). Stationarity refers to a series which doesn't exhibit seasonality or a trend. The data properties such as mean, variance, covariance are independent and not a function of time.
ADF is conducted with the following assumptions:
* `Null Hypothesis` - *Data is non Stationary*
* `Alternate Hypothesis` - *Data is stationary*

Null hypothesis is adopted if the test static is less than the critical value, and p-value is less than 0.05


In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(timeseries,asset):
    print (f'Results of Dickey-Fuller Test: Investment_id{asset} \n')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput,end='')
    print('')

For investment of id 70, we reject null hypothesis for alternate hypothesis. The time series is stationary from the p-value which is less than the significance level of 0.05, and the ADF statistic is less than any critical values

In [ ]:
#  we perform ADF tests for more random investment ids

for i in range(5):
    asset = np.random.randint(0,train.investment_id.nunique())
    d = train.query('investment_id == @asset')
    d = d.set_index('time_id')['target']
    
    
    try:
        adf_test(d,asset)
    except:
        print('err')
        pass

Random investments tested above satisfy the ADF criteria for time stationarity. It can be concluded therefore that the assets in this dataset are stationary.

## **Feature Engineering**

This section outlines steps taken to transform the data in an attempt to improve model performance. The feature engineering tasks are based on exploratory data analysis section. [TIME SERIES ANALYSIS WITH PANDAS](https://ourcodingclub.github.io/tutorials/pandas-time-series/#resampling), [Fitting time series regression models](https://people.duke.edu/~rnau/timereg.html), and [6 Powerful Feature Engineering Techniques For Time Series Data (using Python)](https://www.analyticsvidhya.com/blog/2019/12/6-powerful-feature-engineering-techniques-time-series/) were referenced to guide the process.

### **Create Simulated DateTime Index**

Most of the time series data transformations require a valid DateTime or PeriodTime index. Because the data does not specify one, the time_id column was used to create a dummy variable for Time indexing. It assumes a continous time-period with daily frequency.

In [ ]:
def create_datetime(df,index):
    """ Takes the time_id column of the dataframe to simulate a datetime object
    
     Args: a dataframe and the name of the index with the time-like data

     Returns: dataframe with a datetime index
    
    """
    period=max(df[index]) - min(df[index]) + 2
    dti=pd.date_range(start="2000-01-01", periods=period, freq='D')
    times=[time for time in range(period)]
    data_index_dict=dict(zip(times, dti))
    time_id=df[index].unique()
    df['date_time']=df[index]
    df['date_time'].replace(data_index_dict, inplace=True)
    df['date_time'] =  pd.to_datetime(df['date_time']) 
    
create_datetime(target_sample, 'time_id')
target_sample = target_sample.set_index('date_time')
#train.head()
gc.collect()

### **Autocorrelation**

In [ ]:
def random_autocorrelation_plot (data_frame, time_interval):
    '''
    plots the autocorrelation of 5 random features in a dataframe defined by a time interval
    Args: 
    data_frame: pandas dataframe
    time_interval: pandas datetime format e.g 'W','1w', '2w' 'Y','1y' etc - check pandas.DateTime documentation for full list
    Returns: A graph of autocorrelation over specified time lag
    '''
    import random as r
    random= [r.randint(0,300) for i in range(5)]
    for i in random:
        df = data_frame[f'f_{i}']
        pd.plotting.autocorrelation_plot(df.resample(time_interval).median(), label=(f'feature {i}'))
        plt.title(f"Autocorrelation of feature f_{i} at interval {time_interval}")
    plt.show()

In [ ]:
#Examine how autocorrelation of random features changes with size of our simulated lag times
random_autocorrelation_plot (target_sample, '1w')

random_autocorrelation_plot (target_sample, '1m')

random_autocorrelation_plot (target_sample, '1y')

### **Data Transformations**

Because the data has negative values, we were unable to use some popular mathematical transofrmations used for time-series data e.g log and squaring.

#### **Min-Max Scaling**
The model performance dropped from 0.149 to 0.137 with scaled data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#Normalize the data using Min-Max scaling

sample_x = ['investment_id'] + [f'f_{i}' for i in range(300)]

scaler = MinMaxScaler()

scaled= scaler.fit_transform(target_sample[sample_x])
gc.collect()

scaled



#### **Lagged Dataframe**
The dataframe was lagged with a shift value of 1. Using a plot of autocorrelation lagging the data had the same effect as resampling. No competition submission was made on lagged data alone

In [ ]:
#Copy the dataframe
df_lagged = target_sample.copy()

# del target_sample

#trailing_window_size = 2
df_lagged = pd.concat([df_lagged.shift(1)], axis=1)
df_lagged = df_lagged.fillna(0) #filling with 0's appears to have less of an effect on reducing model performance than mathematical interpolation
df_lagged.head()
gc.collect()

#compare original and lagged data with weekly resampling to check for any change
random_autocorrelation_plot (target_sample, '1w')

random_autocorrelation_plot (df_lagged, '1w')

#### **Differenced Data**
The dataframe was changed so the one-step difference in data was used across both feature and target. This is a method used to make data stationary. Predicition must be made on differenced data and the predictions inverted. We were unable to utilise this as the test_df had no corresponding time-id column to use for setting a DateTime index.

Submiting a model trained on differenced data produced a score of 0.138 which was a drop from 0.149.

In [ ]:
def inverse_diff(actual_df, pred_df):
    '''
    Transforms the differentiated values back
    Args:
        actual dataframe: Values of the columns, numpy array of floats 
        predicted dataframe: Values of the columns, numpy array of floats 
    Returns:
        Dataframe with the predicted values
    '''
    df_res = pred_df.copy()
    columns = actual_df.columns
    for col in columns: 
        df_res[col].replace(actual_df[col].iloc[-1] + df_res[str(col)].cumsum())
    return df_res

In [ ]:
#create a dataset from the differences per increment and pad the first row with 0's
differenced_data= target_sample.diff(axis = 0, periods = 1)
differenced_data=differenced_data.fillna(0)
differenced_data.head()
#del(differenced_data)

#Visualise change in autocorration after differencing
random_autocorrelation_plot (target_sample, '1w')

random_autocorrelation_plot (differenced_data, '1w')

#### Using a lagged and differenced dataframe resulted in a score of  -0.027

### **Resampling the Data**
This would result in a smaller dataframe and utilise medians across a 7 time-step lag. The theory was to reduce noise without losing variation. The autocorrelation plot is smoother but stil variable.


In [ ]:
#Resample dataframe by averaging to period=7 
for column in target_sample.columns:
    resampled_train=target_sample
    resampled_train[column]=resampled_train[column].resample('W').median()
    resampled_train[column]=resampled_train[column].interpolate(method ='linear', limit_direction ='backward')

#Compare the autocorrelation in the new dataframe with original
random_autocorrelation_plot (target_sample, '1w')

random_autocorrelation_plot (resampled_train, '1w')

### **Principal Component Analysis**
The goal is to reduce the dimensionality of features fed into the neural network.

In [ ]:
# installing and importing the dependencies
!pip install pca
from pca import pca

In [ ]:
# pca
def dim_red(df):
    """ This function extracts the principal components of the features of the data sets
    """
    features = [f'f_{i}' for i in range(300)]
    df_ = df[features]
    
    pca()
    pca_model = pca(n_components=0.98) # explained variance 90%
    results = pca_model.fit_transform(df_)
    
    del df_
    
    return results,pca_model

In [ ]:
#  the function obtains the main principal components
target_sample = train.sample(frac = 0.10)

results,pca_model = dim_red(target_sample)

In [ ]:
results.keys()

In [ ]:
#plot the scree plot to show the components with the most variance in the data set
plt.figure(figsize=(10,5))
plt.title('Cumulative Explained Variance vs Principal Components',fontweight='bold',fontsize=12)
plt.plot(np.cumsum(results['variance_ratio']))
plt.xlabel('Num of Components')
plt.ylabel('Cumulative explained variance')
plt.show()

In [ ]:
# plot components/variance
%matplotlib inline
pca_model.plot(figsize=(10,8))
plt.tight_layout()
plt.show()

From the plots above, 250 components has an explained variance of 98 percent.The principal components are loaded into the model as inputs. The results from the model after PCA is -15.3, and therefore not suitable for this challenge. PCA reduces 50 features only, so it doesn't solve the problem of dimensionality.

### **Feature Engineering Summary**

More tests are needed to examine the data inorder to apply feature extraction and selection criteria to improve model performance. We were unable to leverage many feature engineering tools particularly the [scikit](https://scikit-learn.org/stable/auto_examples/applications/plot_cyclical_feature_engineering.html) library  due to computational limits arising from the size of the dataframe.

In [ ]:
# free memory from dataframes in the previous section
del target_sample
del scaled
del resampled_train
del differenced_data
del df_lagged


## **Model Training**
Our baseline model is a Dense Neural Network that uses the swish non-linearity function. Swish is a smooth, non-monotonic function. The data is split into folds with StratifiedKFold from Sklearn, and later split into train and validation sets during training. The four models are trained on this data by looping over the folds. StratifiedKFold cross-validation object is a variation of KFold that returns stratified folds made by preserving the percentage of samples for each class.

The best weights of each model is saved during training and appended into a list for inference. For each fold, the model is trained for  30 epochs. Validation loss is monitored, and early stopping used used to avoid overfitting. Compute resources are optimized by deleting train and validation dataframes after every fold of training.
Validation set is used to evaluate model performance after training. The metrics are Root mean square and Pearson correlation.

In [ ]:
# investment ids
investment_id = train["investment_id"]

# targets
y = train["target"]

# features
train = train.drop(['investment_id','time_id','target'],axis=1)

### **IntegerLookup layer for investment_id**

This is a preprocessing layer which maps integer features to contiguous ranges. During adapt(), the layer will analyze a data set, determine the frequency of individual integer tokens, and create a vocabulary from them.

In [13]:
%%time
#IntegerLookup layer for investment_id 
investment_ids = list(investment_id.unique())
investment_id_size = len(investment_ids) + 1
investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)
investment_id_lookup_layer.adapt(pd.DataFrame({"investment_ids":investment_ids}))

2022-03-06 18:14:10.955982: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


CPU times: user 342 ms, sys: 27.4 ms, total: 369 ms
Wall time: 462 ms


### **Make Tensorflow Dataset**

In [14]:
#data preparation for loading

def preprocess(X, y):
    return X, y

def make_dataset(feature, investment_id, y, batch_size=1024, mode="train"):
    
    """The function creates a source dataset from input data.
    Apply dataset transformations to preprocess the data.
    Iterate over the dataset and process the elements.
    Iteration happens in a streaming fashion, 
    so the full dataset does not need to fit into memory."""
    
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature), y))
    ds = ds.map(preprocess)
    if mode == "train":
        ds = ds.shuffle(4096)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

def t(a):
  """For testing: generate a float64 tensor from anything."""
  return tf.constant(a, dtype=tf.float64)

def tmean(x, axis=-1):
  """Arithmetic mean of a tensor over some axis, default last."""
  x = tf.convert_to_tensor(x)
  sum = tf.reduce_sum(x, axis=axis)
  n = tf.cast(tf.shape(x)[axis], x.dtype)
  return sum / n

def correlationMetric(x, y, axis=-2):
  """Metric returning the Pearson correlation coefficient of two tensors over some axis, default -2."""
  x = tf.convert_to_tensor(x)
  y = math_ops.cast(y, x.dtype)
  n = tf.cast(tf.shape(x)[axis], x.dtype)
  xsum = tf.reduce_sum(x, axis=axis)
  ysum = tf.reduce_sum(y, axis=axis)
  xmean = xsum / n
  ymean = ysum / n
  xvar = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
  yvar = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
  cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
  corr = cov / tf.sqrt(xvar * yvar)
  return tf.constant(1.0, dtype=x.dtype) - corr


def correlationLoss(x,y, axis=-2):
  """Loss function that maximizes the pearson correlation coefficient between the predicted values and the labels,
  while trying to have the same mean and variance"""
  x = tf.convert_to_tensor(x)
  y = math_ops.cast(y, x.dtype)
  n = tf.cast(tf.shape(x)[axis], x.dtype)
  xsum = tf.reduce_sum(x, axis=axis)
  ysum = tf.reduce_sum(y, axis=axis)
  xmean = xsum / n
  ymean = ysum / n
  xsqsum = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
  ysqsum = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
  cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
  corr = cov / tf.sqrt(xsqsum * ysqsum)
  # absdif = tmean(tf.abs(x - y), axis=axis) / tf.sqrt(yvar)
  sqdif = tf.reduce_sum(tf.math.squared_difference(x, y), axis=axis) / n / tf.sqrt(ysqsum / n)
  # meandif = tf.abs(xmean - ymean) / tf.abs(ymean)
  # vardif = tf.abs(xvar - yvar) / yvar
  # return tf.convert_to_tensor( K.mean(tf.constant(1.0, dtype=x.dtype) - corr + (meandif * 0.01) + (vardif * 0.01)) , dtype=tf.float32 )
  return tf.convert_to_tensor( K.mean(tf.constant(1.0, dtype=x.dtype) - corr + (0.01 * sqdif)) , dtype=tf.float32 )

In [15]:
models = []

#### **Model 1 - baseline model**
The baseline model is a dense layer comprising of ten hidden layers. Swish function introduces non linearity.  Loss optimization is Adam with a learning rate of 0.001, and the performance is evaluated with RMSE. This is common across the models forming the ensemble learning.

In [16]:
%%time

def dnn_baseline_model():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse, correlationMetric])
    return model

# training the model

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(5, shuffle=True, random_state=42)

for index, (train_indices, valid_indices) in enumerate(kfold.split(train, investment_id)):
    X_train, X_val = train.iloc[train_indices], train.iloc[valid_indices]
    investment_id_train = investment_id[train_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[valid_indices]
    investment_id_val = investment_id[valid_indices]
    train_ds = make_dataset(X_train, investment_id_train, y_train)
    valid_ds = make_dataset(X_val, investment_id_val, y_val, mode="valid")
    model = dnn_baseline_model()
    checkpoint = keras.callbacks.ModelCheckpoint(f"model_{index}", save_best_only=True)
    early_stop = keras.callbacks.EarlyStopping(patience=10)
    history = model.fit(train_ds, epochs=30, validation_data=valid_ds, callbacks=[checkpoint, early_stop])
    model = keras.models.load_model(f"model_{index}")
    models.append(model)
    
    pearson_score = stats.pearsonr(model.predict(valid_ds).ravel(), y_val.values)[0]
    print('Pearson:', pearson_score)
    pd.DataFrame(history.history, columns=["mse", "val_mse"]).plot()
    plt.title("MSE")
    plt.show()
    pd.DataFrame(history.history, columns=["mae", "val_mae"]).plot()
    plt.title("MAE")
    plt.show()
    pd.DataFrame(history.history, columns=["rmse", "val_rmse"]).plot()
    plt.title("RMSE")
    plt.show()
    del investment_id_train
    del investment_id_val
    del X_train
    del X_val
    del y_train
    del y_val
    del train_ds
    del valid_ds
    gc.collect()
    break

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  UserWarning,


Epoch 1/30
2455/2455 [==============================] - 152s 61ms/step - loss: 0.9107 - mse: 0.8314 - mae: 0.6243 - mape: 136914.6719 - rmse: 0.9118 - correlationMetric: 0.8687 - val_loss: 0.8391 - val_mse: 0.8372 - val_mae: 0.6306 - val_mape: 122048.3672 - val_rmse: 0.9150 - val_correlationMetric: 0.8947


2022-03-06 18:17:33.486862: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 2/30
2455/2455 [==============================] - 123s 50ms/step - loss: 0.8315 - mse: 0.8280 - mae: 0.6233 - mape: 150318.0469 - rmse: 0.9099 - correlationMetric: 0.8563 - val_loss: 0.8376 - val_mse: 0.8353 - val_mae: 0.6282 - val_mape: 176800.3594 - val_rmse: 0.9140 - val_correlationMetric: 0.8868
Epoch 3/30
2455/2455 [==============================] - 122s 50ms/step - loss: 0.8303 - mse: 0.8249 - mae: 0.6219 - mape: 148420.6562 - rmse: 0.9082 - correlationMetric: 0.8437 - val_loss: 0.8382 - val_mse: 0.8341 - val_mae: 0.6281 - val_mape: 140315.6406 - val_rmse: 0.9133 - val_correlationMetric: 0.8813
Epoch 4/30
2455/2455 [==============================] - 123s 50ms/step - loss: 0.8285 - mse: 0.8225 - mae: 0.6210 - mape: 149163.3281 - rmse: 0.9069 - correlationMetric: 0.8347 - val_loss: 0.8361 - val_mse: 0.8341 - val_mae: 0.6279 - val_mape: 151375.5312 - val_rmse: 0.9133 - val_correlationMetric: 0.8793
Epoch 5/30
2455/2455 [==============================] - 119s 48ms/step - loss: 

ValueError: Unable to restore custom object of type _tf_keras_metric currently. Please make sure that the layer implements `get_config`and `from_config` when saving. In addition, please use the `custom_objects` arg when calling `load_model()`.

#### **Model 2 - dnn_10fold_model**
Three dense layers are added to the baseline model. To avoid noise modelling and overfitting, drop out layers are added after every four dense layers. The baseline metric and training params are retained.


In [ ]:
%%time
def dnn_10fold_model():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)    
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
   # investment_id_x = layers.Dropout(0.65)(investment_id_x)
   
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.65)(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
   # x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
  #  x = layers.Dropout(0.4)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.75)(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse, correlationMetric])
    return model

# training

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(10, shuffle=True, random_state=42)

for index, (train_indices, valid_indices) in enumerate(kfold.split(train, investment_id)):
    X_train, X_val = train.iloc[train_indices], train.iloc[valid_indices]
    investment_id_train = investment_id[train_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[valid_indices]
    investment_id_val = investment_id[valid_indices]
    train_ds = make_dataset(X_train, investment_id_train, y_train)
    valid_ds = make_dataset(X_val, investment_id_val, y_val, mode="valid")
    model = dnn_10fold_model()
    checkpoint = keras.callbacks.ModelCheckpoint(f"model_{index}", save_best_only=True)
    early_stop = keras.callbacks.EarlyStopping(patience=10)
    history = model.fit(train_ds, epochs=30, validation_data=valid_ds, callbacks=[checkpoint, early_stop])
    model = keras.models.load_model(f"model_{index}")
    models.append(model)
    
    pearson_score = stats.pearsonr(model.predict(valid_ds).ravel(), y_val.values)[0]
    print('Pearson:', pearson_score)
    pd.DataFrame(history.history, columns=["mse", "val_mse"]).plot()
    plt.title("MSE")
    plt.show()
    pd.DataFrame(history.history, columns=["mae", "val_mae"]).plot()
    plt.title("MAE")
    plt.show()
    pd.DataFrame(history.history, columns=["rmse", "val_rmse"]).plot()
    plt.title("RMSE")
    plt.show()
    del investment_id_train
    del investment_id_val
    del X_train
    del X_val
    del y_train
    del y_val
    del train_ds
    del valid_ds
    gc.collect()
    break

#### **Model 3 - dnn_baseline_model2**
The number of nodes in the dense layers are adjusted compared to the baseline model. Drop out layers introduced after every dense layer. Baseline params are retained.

In [ ]:
%%time
def dnn_baseline_model2():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float32)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.5)(investment_id_x)
    investment_id_x = layers.Dense(32, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.5)(investment_id_x)
    #investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.5)(feature_x)
    feature_x = layers.Dense(128, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.5)(feature_x)
    feature_x = layers.Dense(64, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(16, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1)(x)
    output = tf.keras.layers.BatchNormalization(axis=1)(output)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse, correlationMetric])
    return model

# training

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(5, shuffle=True, random_state=42)

for index, (train_indices, valid_indices) in enumerate(kfold.split(train, investment_id)):
    X_train, X_val = train.iloc[train_indices], train.iloc[valid_indices]
    investment_id_train = investment_id[train_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[valid_indices]
    investment_id_val = investment_id[valid_indices]
    train_ds = make_dataset(X_train, investment_id_train, y_train)
    valid_ds = make_dataset(X_val, investment_id_val, y_val, mode="valid")
    model = dnn_baseline_model2()
    checkpoint = keras.callbacks.ModelCheckpoint(f"model_{index}", save_best_only=True)
    early_stop = keras.callbacks.EarlyStopping(patience=10)
    history = model.fit(train_ds, epochs=30, validation_data=valid_ds, callbacks=[checkpoint, early_stop])
    model = keras.models.load_model(f"model_{index}")
    models.append(model)
    
    pearson_score = stats.pearsonr(model.predict(valid_ds).ravel(), y_val.values)[0]
    print('Pearson:', pearson_score)
    pd.DataFrame(history.history, columns=["mse", "val_mse"]).plot()
    plt.title("MSE")
    plt.show()
    pd.DataFrame(history.history, columns=["mae", "val_mae"]).plot()
    plt.title("MAE")
    plt.show()
    pd.DataFrame(history.history, columns=["rmse", "val_rmse"]).plot()
    plt.title("RMSE")
    plt.show()
    del investment_id_train
    del investment_id_val
    del X_train
    del X_val
    del y_train
    del y_val
    del train_ds
    del valid_ds
    gc.collect()
    break


#### **Model 4 - LSTM Model**
For the first 3 models, the problem is considered as a regression problem that can be solved without factoring the effect of time. In this model, LSTM is used to to carry forward important aspects in time which might affect the future outputs. The output of the dense layer to LSTM is normalized to a mean close to zero and standard deviation of one with batch normalization.

In [ ]:
%%time
def lstm_model():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(128, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(32, activation='swish')(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(128, activation='swish')(feature_x)
    feature_x = layers.Dense(64, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(256, activation='swish', kernel_regularizer="l2")(x)
    # x = layers.Dense(256, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    # x = layers.Dense(16, activation='swish', kernel_regularizer="l2")(x)


    x = layers.BatchNormalization(name='batch_norm1')(x)
    x = layers.Dense(256, activation='swish', name='dense1')(x)
    x = layers.Dropout(0.1, name='dropout1')(x)
    x = layers.Reshape((1, -1), name='reshape1')(x)
    x = layers.BatchNormalization(name='batch_norm2')(x)
    x = layers.LSTM(128, return_sequences=True, activation='relu', name='lstm1')(x)
    x = layers.LSTM(16, return_sequences=False, activation='relu', name='lstm2')(x)


    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss=correlationLoss, metrics=['mse', "mae", rmse, correlationMetric])
    return model

# training


from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(5, shuffle=True, random_state=42)

for index, (train_indices, valid_indices) in enumerate(kfold.split(train, investment_id)):
    X_train, X_val = train.iloc[train_indices], train.iloc[valid_indices]
    investment_id_train = investment_id[train_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[valid_indices]
    investment_id_val = investment_id[valid_indices]
    train_ds = make_dataset(X_train, investment_id_train, y_train)
    valid_ds = make_dataset(X_val, investment_id_val, y_val, mode="valid")
    model = lstm_model()
    checkpoint = keras.callbacks.ModelCheckpoint(f"model_{index}", save_best_only=True)
    early_stop = keras.callbacks.EarlyStopping(patience=10)
    history = model.fit(train_ds, epochs=30, validation_data=valid_ds, callbacks=[checkpoint, early_stop])
    model = keras.models.load_model(f"model_{index}")
    models.append(model)
    
    pearson_score = stats.pearsonr(model.predict(valid_ds).ravel(), y_val.values)[0]
    print('Pearson:', pearson_score)
    pd.DataFrame(history.history, columns=["mse", "val_mse"]).plot()
    plt.title("MSE")
    plt.show()
    pd.DataFrame(history.history, columns=["mae", "val_mae"]).plot()
    plt.title("MAE")
    plt.show()
    pd.DataFrame(history.history, columns=["rmse", "val_rmse"]).plot()
    plt.title("RMSE")
    plt.show()
    del investment_id_train
    del investment_id_val
    del X_train
    del X_val
    del y_train
    del y_val
    del train_ds
    del valid_ds
    gc.collect()
    break

### **Model Summary and Architecture**

In [ ]:
#printing the model summary
model = get_model()
#model.summary()

#desplaing the model architecture
#keras.utils.plot_model(model, show_shapes=True)

### **Inference**
The predicted values of the test data is an average of the predictions made by each saved model.

In [ ]:
def preprocess_test(investment_id, feature):
   return (investment_id, feature), 0
def make_test_dataset(feature, investment_id, batch_size=1024):
   ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature)))
   ds = ds.map(preprocess_test)
   ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
   return ds
def inference(models, ds):
   y_preds = []
   for model in models:
       y_pred = model.predict(ds)
       y_preds.append(y_pred)
   return np.mean(y_preds, axis=0)

### **Submission**

In [ ]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test() 
for (test_df, sample_prediction_df) in iter_test:
   ds = make_test_dataset(test_df[features], test_df["investment_id"])
   sample_prediction_df['target'] = inference(models, ds)
   env.predict(sample_prediction_df) 

## **Results**


## **Conclusion**

The highest score achieved was 0.153, and further increment of the ensemble models couldn't improve the results. The task was considered as a regression problem that could be solved with a feed forward neural network. From the EDA, there are so many complexities and noise in the dataset which limits performance. More so, 27 percent of the data is missing. Feature engineering techniques such as PCA, interpolating data points couldn't make it better. Kernel crashes due to limited computer memory impeded training, and feature selection on the dataset. The team members were satisfied with the Kaggle leaderboard score despite the challenges and the short amount of time available for the task.